In [1]:
from generator_based_on_Seattle import *
from parameters import *
from datetime import datetime

In [3]:
# predict traffic

time_delta = datetime.strptime(TO_DATE,"%Y-%m-%d") - datetime.strptime(FROM_DATE,"%Y-%m-%d")
n_periods = math.floor(time_delta.days / UPGRADE_PERIOD)

def generate_for_load(LAMBDA = 100, ALPHAS_LIST = [2]):
	parent_folder: str = 'generated_traffic_data'
	load_folder_path: str = parent_folder + '/' + 'load' + '_' + str(LAMBDA)
	actual_traffic_folder_path: str = load_folder_path + '/' + 'actual_traffic'
	predicted_traffic_folder_path: str = load_folder_path + '/' + 'predicted_traffic'


	# Do not create patent folder 'generated_traffic_data' again
	Path(load_folder_path).mkdir(parents=True, exist_ok=True)
	Path(actual_traffic_folder_path).mkdir(parents=True, exist_ok=True)
	Path(predicted_traffic_folder_path).mkdir(parents=True, exist_ok=True)


	for alpha in ALPHAS_LIST:
		predicted_traffic_alphafolder_path: str = load_folder_path + '/' + 'predicted_traffic' + '/' + 'alpha' + str(alpha)
		Path(predicted_traffic_alphafolder_path).mkdir(parents=True, exist_ok=True)
		
	generate_traffic_based_on_seattle(
	    from_date=FROM_DATE, 
	    to_date=TO_DATE, 
	    first_erlang=LAMBDA, 
	    constant_bitrate=IS_CONSTANT_BITRATE, 
	    aggregation=AGGREGATION, 
	    resampling=RESAMPLING,
	    actual_traffic_folder_path=actual_traffic_folder_path)

	# predict three periods
	columns = ["current_global_time", "source_id", "destination_id", "datarate", "arrival_time", "departure_time"]

	for i in range(n_periods):
		filepath: str = actual_traffic_folder_path + '/' + 'actual_traffic' + '_' + str(UPGRADE_PERIOD * i) + '_' + str(UPGRADE_PERIOD * (i + 1)) + '.txt'
		print(i)

		traffic_df = pd.read_csv(filepath, delimiter='\s\s+', engine='python', header=None, names=columns)
		
	
		for alpha in ALPHAS_LIST:
			predicted_traffic_alphafolder_path: str = load_folder_path + '/' + 'predicted_traffic' + '/' + 'alpha' + str(alpha)
			perdict_traffic_for_next_period( traffic_df,alpha =alpha,predicted_traffic_alphafolder_path=predicted_traffic_alphafolder_path,index=i)

for lamda_0 in [100,150,200,250,300,350]:
	print(lamda_0)
	generate_for_load(lamda_0,[1,3,5,10])

In [4]:
n_connections_differences_overall = []
mean_holding_time_differences_overall = []

for current_period_start in range(0 + UPGRADE_PERIOD, n_periods*UPGRADE_PERIOD, UPGRADE_PERIOD):
    print(current_period_start)
    real_traffic = pd.read_csv(f"traffic_from_day_{current_period_start}.csv")
    real_traffic['holding_time'] = real_traffic['departure_time'] - real_traffic['arrival_time']
    predicted_traffic = pd.read_csv(f"prediction_from_day_{current_period_start}.csv")
    predicted_traffic['holding_time'] = predicted_traffic['departure_time'] - predicted_traffic['arrival_time']

    n_connections_generated = []
    n_connections_predicted = []
    n_connections_differences = []
    mean_holding_time_generated = []
    mean_holding_time_predicted = []
    mean_holding_time_differences = []
    for source_node in range(14):
        for destination_node in range(14):
            if(source_node != destination_node):
                generated_connections = real_traffic.loc[(real_traffic['source_id'] == source_node) & (real_traffic['destination_id'] == destination_node)]
                predicted_connections = predicted_traffic.loc[(predicted_traffic['source_id'] == source_node) & (predicted_traffic['destination_id'] == destination_node)]
                generated_mean_holding_time = generated_connections['holding_time'].mean()
                predicted_mean_holding_time = predicted_connections['holding_time'].mean()
                if(math.isnan(generated_mean_holding_time)):
                    print("{} --> {}".format(source_node,destination_node))
                if(math.isnan(predicted_mean_holding_time)):
                    print("{} --> {}".format(source_node,destination_node))    

                n_connections_generated.append(len(generated_connections))
                n_connections_predicted.append(len(predicted_connections))
                n_connections_differences.append(len(predicted_connections) - len(generated_connections))
                mean_holding_time_generated.append(generated_mean_holding_time)
                mean_holding_time_predicted.append(predicted_mean_holding_time)
                mean_holding_time_differences.append(predicted_mean_holding_time-generated_mean_holding_time)

    n_connections_differences_overall.append(n_connections_differences)
    mean_holding_time_differences_overall.append(mean_holding_time_differences)            


90
180
270
360
450
540
630
720
810
900
990
1080
1170
1260
1350
1440
1530
1620
1710
1800
1890
1980
2070
2160
2250
2340
2430
2520
2610
2700
2790
2880
2970
3060
3150
3240
3330
3420
3510


In [5]:
for i in range(len(n_connections_differences_overall)):
    print(f"n_connections diff {np.mean(n_connections_differences_overall[i])}, holding_time diff {np.mean(mean_holding_time_differences_overall[i])}")

n_connections diff 6.532967032967033, holding_time diff 0.3187610205254084
n_connections diff 3766.8076923076924, holding_time diff 0.2920564264167496
n_connections diff 3242.0604395604396, holding_time diff 0.3366027028262374
n_connections diff 156.06593406593407, holding_time diff 0.5431262444197618
n_connections diff 6539.247252747253, holding_time diff 0.12888177212579593
n_connections diff 222.67032967032966, holding_time diff 0.26650574144330447
n_connections diff 8714.917582417582, holding_time diff 0.3242665404734491
n_connections diff 4428.851648351649, holding_time diff 0.36019538120660477
n_connections diff 9591.703296703297, holding_time diff 0.19030944466485913
n_connections diff 2511.7252747252746, holding_time diff 0.2449412533703576
n_connections diff 6571.917582417583, holding_time diff 0.2126390678497907
n_connections diff 6992.807692307692, holding_time diff 0.39936207388993583
n_connections diff 5933.593406593406, holding_time diff 0.3869877870963915
n_connections d